In [1]:
import pandas as pd
import numpy as np
import random
import itertools
import statsmodels.api as sm

In [2]:
# Charger le fichier Excel dans un DataFrame
df = pd.read_excel('regression_results.xlsx', sheet_name=None)

# Accéder aux données du DataFrame
for sheet_name, sheet_df in df.items():
    print(f"Feuille : {sheet_name}")
    print(sheet_df)
    print()

Feuille : WTI
      Unnamed: 0                             Variables_explicatives  \
0              0                                     ['RUSSIA GDP']   
1              1                         ['USA INFLATION RATE CPI']   
2              2                             ['USA INTEREST RATES']   
3              3                      ['USA INDUSTRIAL PRODUCTION']   
4              4                               ['RUSSIA RTS INDEX']   
...          ...                                                ...   
8186        8186  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   
8187        8187  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   
8188        8188  ['RUSSIA GDP', 'USA INTEREST RATES', 'USA INDU...   
8189        8189  ['USA INFLATION RATE CPI', 'USA INTEREST RATES...   
8190        8190  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   

                                           Coefficients  \
0     const         65.080373\nRUSSIA GDP    20.0484...   
1     const    

In [4]:
# Dictionary to store the best model for each dependent variable
best_models = {}

# Loop over the dependent variables
for dependent_var, results_table in df.items():
    # Find the row with the highest adjusted R-squared
    best_model_row = results_table.loc[results_table['Adjusted_R_squared'].idxmax()]

    # Save the best model to the dictionary
    best_models[dependent_var] = {
        'Variables_explicatives': best_model_row['Variables_explicatives'],
        'Coefficients': best_model_row['Coefficients'],
        'Erreurs_standard': best_model_row['Erreurs_standard'],
        'P-value': best_model_row['P-value'],
        'Autocorr_test': best_model_row['Autocorr_test'],
        'Hetero_test': best_model_row['Hetero_test'],
        'Multicollinearity_Fstat': best_model_row['Multicollinearity_Fstat'],
        'Multicollinearity_pvalue': best_model_row['Multicollinearity_pvalue'],
        'Adjusted_R_squared': best_model_row['Adjusted_R_squared']
    }

# Print the best models for each dependent variable
for dependent_var, best_model in best_models.items():
    print(f"Meilleur modèle pour la variable dépendante : {dependent_var}")
    print(best_model)
    print()

Meilleur modèle pour la variable dépendante : WTI
{'Variables_explicatives': "['USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA RETAIL SALES', 'USA IMPORT PRICES', 'RUSSIA CSI']", 'Coefficients': 'const                       -87.412886\nUSA INFLATION RATE CPI        4.187682\nUSA INTEREST RATES           -2.501490\nUSA INDUSTRIAL PRODUCTION     1.439646\nRUSSIA RTS INDEX             -0.034872\nRUSSIA INFLATION RATE         1.033221\nRUSSIA TRADE BALANCE          1.699234\nUSA UNEMPLOYMENT RATE         1.524072\nRUSSIA RETAIL SALES          -0.202413\nUSA IMPORT PRICES            -1.782786\nRUSSIA CSI                    1.931766\ndtype: float64', 'Erreurs_standard': 'const                        14.237165\nUSA INFLATION RATE CPI        0.635761\nUSA INTEREST RATES            1.400141\nUSA INDUSTRIAL PRODUCTION     0.717141\nRUSSIA RTS INDEX              0.006

In [5]:
import pandas as pd

# Charger le fichier Excel dans un DataFrame
df = pd.read_excel('regression_results.xlsx', sheet_name=None)

# Loop through the results dictionary
for dependent_var, results_table in df.items():
    # Find the row with the highest R-squared value
    best_model_row = results_table.loc[results_table['Adjusted_R_squared'].idxmax()]

    # Extract the variables and R-squared value
    best_explanatory_vars = best_model_row['Variables_explicatives']
    best_adj_r_squared = best_model_row['Adjusted_R_squared']

    # Get the index (line number) of the best model row
    line_number = best_model_row.name + 1  # Adding 1 to convert from zero-based index to line number

    # Print the results
    print(f"Variable dépendante : {dependent_var}")
    print(f"Ligne : {line_number}")
    print(f"Variables explicatives : {best_explanatory_vars}")
    print(f"R² ajusté : {best_adj_r_squared}")
    print()


Variable dépendante : WTI
Ligne : 8057
Variables explicatives : ['USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA RETAIL SALES', 'USA IMPORT PRICES', 'RUSSIA CSI']
R² ajusté : 0.7576463039142344

Variable dépendante : Henry Hub Nat Gas
Ligne : 6988
Variables explicatives : ['USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA TRADE BALANCE', 'RUSSIA RETAIL SALES', 'RUSSIA EXPORT PRICES', 'USA IMPORT PRICES', 'RUSSIA CSI']
R² ajusté : 0.4689863859737227

Variable dépendante : Corn
Ligne : 7348
Variables explicatives : ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA EXPORT PRICES', 'RUSSIA CSI']
R² ajusté : 0.6854199527025201

Variable dépendante : Gold
Ligne : 8180
Variables explicatives : ['RUSSIA GDP', 'USA IN

In [6]:
# Dictionary to store the presence of heteroscedasticity, multicollinearity, and serial autocorrelation for each best model
diagnostics_dict = {}

# Loop over the dependent variables and their best models
for dependent_var, best_model in best_models.items():
    # Get the necessary results from the best model
    hetero_test = best_model['Hetero_test']
    multicollinearity_pvalue = best_model['Multicollinearity_pvalue']
    autocorr_test = best_model['Autocorr_test']

    # Determine the presence of heteroscedasticity
    hetero_present = hetero_test < 0.05  # Check if the p-value is below the significance level (e.g., 0.05)

    # Determine the presence of multicollinearity
    multicollinear_present = multicollinearity_pvalue < 0.05

    # Determine the presence of serial autocorrelation
    autocorr_present = autocorr_test < 2 or autocorr_test > 4  # Check if the Durbin-Watson statistic is significantly different from 2

    # Save the diagnostic results to the diagnostics dictionary
    diagnostics_dict[dependent_var] = {
        'Heteroscedasticity': hetero_present,
        'Multicollinearity': multicollinear_present,
        'Serial_Autocorrelation': autocorr_present
    }

# Print the diagnostic results for each dependent variable
for dependent_var, diagnostics in diagnostics_dict.items():
    print(f"Diagnostics for the dependent variable: {dependent_var}")
    print("Heteroscedasticity:", diagnostics['Heteroscedasticity'])
    print("Multicollinearity:", diagnostics['Multicollinearity'])
    print("Serial Autocorrelation:", diagnostics['Serial_Autocorrelation'])
    print()

Diagnostics for the dependent variable: WTI
Heteroscedasticity: False
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Henry Hub Nat Gas
Heteroscedasticity: True
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Corn
Heteroscedasticity: True
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Gold
Heteroscedasticity: True
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Sugar 11
Heteroscedasticity: False
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Cocoa
Heteroscedasticity: False
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Live Cattle
Heteroscedasticity: False
Multicollinearity: True
Serial Autocorrelation: True

Diagnostics for the dependent variable: Platinium
Heteroscedasticity: False
Multicollinearity: True
Seri

In [7]:
# Create a DataFrame from the diagnostics dictionary
diagnostics_df = pd.DataFrame.from_dict(diagnostics_dict, orient='index')

# Count the number of True and False values for each test
count_df = diagnostics_df.apply(pd.value_counts)

# Print the count table
print("Summary of Diagnostic Tests:")
print(count_df)

#if the value for the heteroscedasticity test is True, it means that there is presence of heteroscedasticity in the model
# For the multicollinearity test, if the value is True, it means that there is presence of multicollinearity in the model
#For the serial autocorrelation test, if the value is True, it means that there is serial autocorrelation in the model

Summary of Diagnostic Tests:
       Heteroscedasticity  Multicollinearity  Serial_Autocorrelation
False                   7                NaN                     NaN
True                    3               10.0                    10.0


In [1]:
#From here is about SQL export procedure
import sqlite3
connection = sqlite3.connect('C:\\Users\\User\\Desktop\\AQM My SQLite Database AssetPr. and Macr.Var - Αντιγραφή.db')
cursor = connection.cursor()

In [2]:
#SQLITE Datatypes

#NUMERIC
#INTEGER
#REAL
#TEXT
#BLOBS (Binary Large Objects)
#NULL

In [3]:
cursor.execute("DROP TABLE IF EXISTS Results_1")
cursor.execute("CREATE TABLE RESULTS_1(Dependent_Variable TEXT, Line INTEGER, Explanatory_Variables TEXT, Adjusted_R_Squared REAL)")

In [4]:
connection.commit()

In [5]:
# Define your records
records = [
    ('WTI', 8057, 'USA INFLATION RATE CPI;USA INTEREST RATES;USA INDUSTRIAL PRODUCTION;RUSSIA RTS INDEX;RUSSIA INFLATION RATE;RUSSIA TRADE BALANCE;USA UNEMPLOYMENT RATE;RUSSIA RETAIL SALES;USA IMPORT PRICES;RUSSIA CSI', 0.7576463039142344),
    ('Henry Hub Nat Gas', 6988, ';'.join(['USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA TRADE BALANCE', 'RUSSIA RETAIL SALES', 'RUSSIA EXPORT PRICES', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.4689863859737227),
    ('Corn', 7348, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA EXPORT PRICES', 'RUSSIA CSI']), 0.6854199527025201),
    ('Gold', 8180, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'USA GDP', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA RETAIL SALES', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.6545742018114808),
    ('Sugar 11', 7973, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'RUSSIA RTS INDEX', 'USA GDP', 'RUSSIA INFLATION RATE', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA EXPORT PRICES', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.4785516011467238),
    ('Cocoa', 4217, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'RUSSIA RTS INDEX', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA EXPORT PRICES']), 0.6106505147691611),
    ('Live Cattle', 7829, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'USA GDP', 'RUSSIA INFLATION RATE', 'USA UNEMPLOYMENT RATE', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.7404531889622552),
    ('Platinium', 4181, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'RUSSIA RTS INDEX', 'USA GDP', 'RUSSIA INFLATION RATE', 'USA UNEMPLOYMENT RATE']), 0.8070712988743498),
    ('Zinc', 8117, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'USA GDP', 'RUSSIA TRADE BALANCE', 'USA UNEMPLOYMENT RATE', 'RUSSIA RETAIL SALES', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.6583229687585329),
    ('Lead', 7162, ';'.join(['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA INTEREST RATES', 'USA INDUSTRIAL PRODUCTION', 'RUSSIA RTS INDEX', 'RUSSIA TRADE BALANCE', 'RUSSIA RETAIL SALES', 'USA IMPORT PRICES', 'RUSSIA CSI']), 0.5155316702937889)
]


# Prepare SQL query
sql_query = "INSERT INTO Results_1 VALUES (?, ?, ?, ?)"


# Execute the SQL command
cursor.executemany(sql_query, records)

# Commit your changes
connection.commit()

In [6]:
# Prepare SQL query to fetch all data from the table
sql_query = "SELECT * FROM Results_1"

# Execute the SQL command
cursor.execute(sql_query)

# Fetch all the rows in a list of lists
results = cursor.fetchall()

for row in results:
    print(row)

('WTI', 8057, 'USA INFLATION RATE CPI;USA INTEREST RATES;USA INDUSTRIAL PRODUCTION;RUSSIA RTS INDEX;RUSSIA INFLATION RATE;RUSSIA TRADE BALANCE;USA UNEMPLOYMENT RATE;RUSSIA RETAIL SALES;USA IMPORT PRICES;RUSSIA CSI', 0.7576463039142344)
('Henry Hub Nat Gas', 6988, 'USA INTEREST RATES;USA INDUSTRIAL PRODUCTION;RUSSIA RTS INDEX;RUSSIA TRADE BALANCE;RUSSIA RETAIL SALES;RUSSIA EXPORT PRICES;USA IMPORT PRICES;RUSSIA CSI', 0.4689863859737227)
('Corn', 7348, 'RUSSIA GDP;USA INFLATION RATE CPI;USA INDUSTRIAL PRODUCTION;RUSSIA RTS INDEX;RUSSIA INFLATION RATE;RUSSIA TRADE BALANCE;USA UNEMPLOYMENT RATE;RUSSIA EXPORT PRICES;RUSSIA CSI', 0.6854199527025201)
('Gold', 8180, 'RUSSIA GDP;USA INFLATION RATE CPI;USA INTEREST RATES;USA INDUSTRIAL PRODUCTION;RUSSIA RTS INDEX;USA GDP;RUSSIA INFLATION RATE;RUSSIA TRADE BALANCE;USA UNEMPLOYMENT RATE;RUSSIA RETAIL SALES;USA IMPORT PRICES;RUSSIA CSI', 0.6545742018114808)
('Sugar 11', 7973, 'RUSSIA GDP;USA INFLATION RATE CPI;RUSSIA RTS INDEX;USA GDP;RUSSIA INFLAT

In [7]:
import sqlite3
connection = sqlite3.connect('C:\\Users\\User\\Desktop\\AQM My SQLite Database AssetPr. and Macr.Var - Αντιγραφή.db')
cursor = connection.cursor()

In [8]:
#SQLITE Datatypes

#NUMERIC
#INTEGER
#REAL
#TEXT
#BLOBS (Binary Large Objects)
#NULL

In [9]:
connection.commit()

In [11]:
# Boolean values (True/False) are usually stored as integers, where True is represented by 1 and False is represented by 0. However we want to depict the true or falseso we save it like a text(a text string.)
cursor.execute("DROP TABLE IF EXISTS Results_2")
cursor.execute("CREATE TABLE RESULTS_2 (Dependent_Variable TEXT, Heteroscedasticity TEXT, Multicollinearity TEXT, Serial_Autocorrelation TEXT)")

In [12]:
# Define your records
diagnostics = [
    ('WTI', 'False', 'True', 'True'),
    ('Henry Hub Nat Gas', 'True', 'True', 'True'),
    ('Corn', 'True', 'True', 'True'),
    ('Gold', 'True', 'True', 'True'),
    ('Sugar 11', 'False', 'True', 'True'),
    ('Cocoa', 'False', 'True', 'True'),
    ('Live Cattle', 'False', 'True', 'True'),
    ('Platinium', 'False', 'True', 'True'),
    ('Zinc', 'False', 'True', 'True'),
    ('Lead', 'False', 'True', 'True')
]

# Prepare SQL query
sql_query = "INSERT INTO Results_2 VALUES (?, ?, ?, ?)"

# Execute the SQL command
cursor.executemany(sql_query, diagnostics)

# Commit your changes
connection.commit()

In [13]:
# Prepare SQL query to fetch all data from the table
sql_query = "SELECT * FROM Results_2"

# Execute the SQL command
cursor.execute(sql_query)

# Fetch all the rows in a list of lists
results = cursor.fetchall()

for row in results:
    print(row)

('WTI', 'False', 'True', 'True')
('Henry Hub Nat Gas', 'True', 'True', 'True')
('Corn', 'True', 'True', 'True')
('Gold', 'True', 'True', 'True')
('Sugar 11', 'False', 'True', 'True')
('Cocoa', 'False', 'True', 'True')
('Live Cattle', 'False', 'True', 'True')
('Platinium', 'False', 'True', 'True')
('Zinc', 'False', 'True', 'True')
('Lead', 'False', 'True', 'True')


In [14]:
import sqlite3
connection = sqlite3.connect('C:\\Users\\User\\Desktop\\AQM My SQLite Database AssetPr. and Macr.Var - Αντιγραφή.db')
cursor = connection.cursor()

In [15]:
#SQLITE Datatypes

#NUMERIC
#INTEGER
#REAL
#TEXT
#BLOBS (Binary Large Objects)
#NULL

In [16]:
# Boolean values (True/False) are usually stored as integers, where True is represented by 1 and False is represented by 0. However we want to depict the true or false so we save it like a textcursor.execute("DROP TABLE IF EXISTS Results_3")
cursor.execute("DROP TABLE IF EXISTS Results_3")
cursor.execute("""
CREATE TABLE RESULTS_3(
    TestCondition TEXT,
    Heteroscedasticity INTEGER,
    Multicollinearity INTEGER,
    Serial_Autocorrelation INTEGER
);
""")

In [17]:
# Define your records
records = [
    ('False', 7, None, None),
    ('True', 3, 10, 10),
]

# Prepare SQL query
sql_query = """
INSERT INTO Results_3 (TestCondition, Heteroscedasticity, Multicollinearity, Serial_Autocorrelation) 
VALUES (?, ?, ?, ?)
"""

# Execute the SQL command
cursor.executemany(sql_query, records)

# Commit your changes
connection.commit()

In [18]:
# Prepare SQL query to fetch all data from the table
sql_query = "SELECT * FROM Results_3"

# Execute the SQL command
cursor.execute(sql_query)

# Fetch all the rows in a list of lists
results = cursor.fetchall()

for row in results:
    print(row)

('False', 7, None, None)
('True', 3, 10, 10)


In [19]:
# From here is the procedure where it read them from SQL to connect them with python
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(r'C:\Users\User\Desktop\Results_4.db')

# Get a list of all the table names in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# tables is a list of tuples, where each tuple has one item (the table name).
# We will flatten this to a simple list of table names.
tables = [table[0] for table in tables]

# Load each table into a DataFrame and store them all in a dictionary
dataframes = {}
for table in tables:
    query = f'SELECT * FROM "{table}"'
    dataframes[table] = pd.read_sql_query(query, conn)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Cocoa',), ('Corn',), ('Henry Hub Nat Gas',), ('Gold',), ('Lead',), ('Live_Cattle',), ('Platinium',), ('Sugar_11',), ('WTI',), ('Zinc',)]


In [20]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(r'C:\Users\User\Desktop\Results_4.db')

# List of table names
table_names = ['Cocoa', 'Corn', 'Henry Hub Nat Gas', 'Gold', 'Lead', 'Live_Cattle', 'Platinium', 'Sugar_11', 'WTI', 'Zinc']

dataframes = {} # We will store each dataframe in this dictionary, with table names as keys

for table_name in table_names:
    # Create a query to select all from your table
    query = f'SELECT * FROM "{table_name}"'
    
    # Execute the query and load the data into a DataFrame
    data = pd.read_sql_query(query, conn)

    # Now 'data' is a pandas DataFrame containing all data from your table
    dataframes[table_name] = data

# Don't forget to close the connection when you're done:
conn.close()

for table_name, dataframe in dataframes.items():
    print(f"Data for {table_name}:")
    print(dataframe)
    print("\n")  # print a newline for readability

Data for Cocoa:
      Line                             Variables_explicatives  \
0        0                                     ['RUSSIA GDP']   
1        1                         ['USA INFLATION RATE CPI']   
2        2                             ['USA INTEREST RATES']   
3        3                      ['USA INDUSTRIAL PRODUCTION']   
4        4                               ['RUSSIA RTS INDEX']   
...    ...                                                ...   
8186  8186  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   
8187  8187  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   
8188  8188  ['RUSSIA GDP', 'USA INTEREST RATES', 'USA INDU...   
8189  8189  ['USA INFLATION RATE CPI', 'USA INTEREST RATES...   
8190  8190  ['RUSSIA GDP', 'USA INFLATION RATE CPI', 'USA ...   

                                           Coefficients  \
0     const         2569.737703\nRUSSIA GDP   -1631....   
1     const                     2438.557935\nUSA INF...   
2     const               